In [1]:
import numpy as np
import pandas as pd
import math
from datetime import time
import CMAPSAuxFunctions

from data_handler_VALVE import ValveDataHandler
from tunable_model import SequenceTunableModelRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn import metrics

from keras.models import Sequential, Model
from keras.layers import Dense, Input, Dropout, Reshape, Conv2D, Flatten, MaxPooling2D
from keras.optimizers import Adam, SGD
from keras.callbacks import LearningRateScheduler
from keras import backend as K
from keras import regularizers

Using TensorFlow backend.


<h1> Create Data Handler </h1>

In [2]:
features = ['timestamp', 'externalControllerOutput', 'undisturbedMediumFlow',
            'pressureValveInlet', 'pressureValveOutlet', 'mediumTemperature',
            'rodDisplacement', 'disturbedMediumFlow', 'selectedFault',
            'faultType', 'faultIntensity']
selected_indices = np.array([2,4,5,6,7,8])
selected_features = list(features[i] for i in selected_indices-1)

window_size = 30
window_stride = 1

# min = 2018-02-14 18:59:20
# max = 2018-08-19 18:28:20
time_start = "2018-02-14 18:59:20"
time_end = "2018-08-19 18:28:20"

# Either classification or regression
problem = 'classification'
#problem = 'regression'

vHandler = ValveDataHandler(time_start, time_end, selected_features = selected_features,
                            sequence_length = window_size, sequence_stride = window_stride,
                            problem = problem)

init


<h1> Keras Model </h1>

In [3]:
#np.set_printoptions(threshold=np.nan)

K.clear_session()
lambda_regularization = 0.20

def create_ANN_model(input_shape, problem):
    
    #Create a sequential model
    model = Sequential()
    
    #Add the layers for the model
    model.add(Dense(20, input_dim = input_shape, activation = 'relu', kernel_initializer = 'glorot_normal', 
                    kernel_regularizer = regularizers.l2(lambda_regularization), name = 'fc1'))
    
    if (problem == 'classification'):
        model.add(Dense(2, activation = 'softmax', name = 'out'))
    elif (problem == 'regression'):
        model.add(Dense(1, activation = 'linear', name = 'out'))
        
    #model.add(Dense(1, activation = 'softmax', name = 'out'))
    
    return model

<h1>Tunable Model </h1>

In [4]:
scaler = MinMaxScaler(feature_range = (-1, 1))
#scaler = StandardScaler()

In [5]:
def get_compiled_model(shape, problem):
    
    K.clear_session()
    
    if (problem == 'classification'):
        # Parameters for the model
        optimizer = SGD(lr = 0.01, momentum = 0.0, decay = 0.0, nesterov = False)
        loss_function = 'categorical_crossentropy'
        metrics = ['accuracy']
    elif (problem == 'regression'):
        # Parameters for the model
        optimizer = Adam(lr = 0.001, beta_1 = 0.9, beta_2 = 0.999, epsilon = None, decay = 0.0, amsgrad = False)
        loss_function = 'mean_squared_error'
        metrics = ['mse']
        
    model = None
    
    # Create and compile the model
    model = create_ANN_model(shape, problem)
    model.compile(optimizer = optimizer, loss = loss_function, metrics = metrics)
    
    return model

In [6]:
num_features = len(selected_features)
input_shape = num_features * window_size
print(input_shape)

model = get_compiled_model(input_shape, problem)
tModel = SequenceTunableModelRegression('ANN_Model', model, lib_type = 'keras', data_handler = vHandler)

tModel.data_handler.data_scaler = None
tModel.data_scaler = scaler

180


<h1> Loading Data from MySQL Database </h1>

In [7]:
#vHandler.connect_to_db("remoteAdmin","remoteAdmin","169.236.181.40:3306","damadics")

In [8]:
# Extract data from database
# vHandler.extract_data_from_db()
# vHandler.extract_data_from_db()

In [9]:
# Extract data from .csv file. If loading from .csv, don't forget to uncomment .extract_data_from_db() in data_handler_VALVE'''
vHandler._df = pd.read_csv('valve_dataset.csv', sep = ',')
for i in range(1,9):
    vHandler._df[vHandler._df.columns[-i]] = vHandler._df[vHandler._df.columns[-i-1]]
vHandler._df['undisturbedMediumFlow'] = pd.np.nan

# adds a new feature called "Status" on whether the valve is broken '1' or not broken '0'
#vHandler._df['status'] = vHandler._df['selectedFault'].apply(lambda valve: 0 if valve == 20 else 1)
# this adds a column to vHandler._X
#vHandler._X = vHandler._df[selected_features]
#vHandler._X.insert(loc = 0, column = 'id', value = np.arange(len(vHandler._df)))
# vHandler._X = vHandler._X.values

# vHandlezr._y = vHandler._df['status'].values
print(vHandler._df.head())

   id            timestamp  externalControllerOutput  undisturbedMediumFlow  \
0   1  2018-02-14 18:59:20                  0.500000                    NaN   
1   2  2018-02-14 19:00:20                  0.366043                    NaN   
2   3  2018-02-14 19:01:20                  0.732444                    NaN   
3   4  2018-02-14 19:02:20                  0.257854                    NaN   
4   5  2018-02-14 19:03:20                  0.659356                    NaN   

   pressureValveInlet  pressureValveOutlet  mediumTemperature  \
0            0.876840             0.650832           0.214553   
1            0.850576             0.643717           0.215373   
2            0.849537             0.647472           0.214781   
3            0.849203             0.648064           0.214384   
4            0.848475             0.654271           0.214159   

   rodDisplacement  disturbedMediumFlow  selectedFault  faultType  \
0         0.001590             1.000000             20          1

In [10]:
#vHandler._df['id'] = np.arange(1, vHandler._df.shape[0] + 1)
#print(vHandler._df)

In [11]:
vHandler._X = vHandler._df[selected_features].values

if (problem == 'classification'):
    # 0; [1; 0] = not broken
    # 1; [0; 1] = broken
    vHandler._df['status'] = vHandler._df['selectedFault'].apply(lambda valve: 0 if valve == 20 else 1)
    vHandler._y = vHandler._df["status"].values
elif (problem == 'regression'):
    vHandler._y = vHandler._df["selectedFault"].values
        
# vHandler._X = vHandler._df[selected_features].values
# vHandler._y = vHandler._df["selectedFault"].values
#vHandler._y = list(map(lambda valve: 0 if valve == 20 else 1, vHandler._y))
#vHandler._y = vHandler._df["status"].values

In [12]:
print(vHandler._X[0:5])
print(vHandler._y[0:5])
# print(vHandler._y)

[[0.5      0.87684  0.650832 0.214553 0.00159  1.      ]
 [0.366043 0.850576 0.643717 0.215373 0.369939 1.      ]
 [0.732444 0.849537 0.647472 0.214781 0.732446 0.222181]
 [0.257854 0.849203 0.648064 0.214384 0.239649 0.99759 ]
 [0.659356 0.848475 0.654271 0.214159 0.67503  0.283687]]
[0 0 0 0 0]


In [13]:
#output_one_hot_matrix = tModel.data_handler.one_hot_encode(tModel.data_handler._df.shape[0])
#print(output_one_hot_matrix[216])

In [14]:
# Finds valve samples in the dataset
tModel.data_handler._X, tModel.data_handler._num_samples = tModel.data_handler.find_samples(tModel.data_handler._X)
tModel.data_handler._y, _ = tModel.data_handler.find_samples(tModel.data_handler._y)

265100
Splitting into samples: 0:00:01.019478
counter: 130
265100
Splitting into samples: 0:00:01.215202
counter: 130


In [15]:
print("Amount of num_samples:", tModel.data_handler._num_samples)
for i in range(5):
    print("X[{}].shape: {}".format(i, tModel.data_handler._X[i].shape))
    print("y[{}].shape: {}".format(i, tModel.data_handler._y[i].shape))
    print("{}".format(tModel.data_handler._y[i][0]))
    print("{}\n".format(tModel.data_handler._y[i][-1]))

Amount of num_samples: 130
X[0].shape: (216, 6)
y[0].shape: (216, 1)
[0]
[0]

X[1].shape: (436, 6)
y[1].shape: (436, 1)
[1]
[0]

X[2].shape: (1048, 6)
y[2].shape: (1048, 1)
[1]
[0]

X[3].shape: (1382, 6)
y[3].shape: (1382, 1)
[1]
[0]

X[4].shape: (710, 6)
y[4].shape: (710, 1)
[1]
[0]



In [16]:
tModel.data_handler.train_cv_test_split(cross_validation_ratio = 0.2, test_ratio = 0.2)

Train, cv, and test splitting: 0:00:00.000803


In [17]:
print(tModel.data_handler._X_crossVal_list[0])

[[4.84302e-01 8.49100e-01 6.50041e-01 2.15571e-01 9.99730e-01 0.00000e+00]
 [3.66043e-01 8.48995e-01 6.53369e-01 2.14565e-01 1.00000e+00 1.78400e-03]
 [7.32444e-01 8.49803e-01 6.53430e-01 2.13269e-01 1.00000e+00 0.00000e+00]
 [2.57854e-01 8.51221e-01 6.58088e-01 2.18497e-01 9.99215e-01 0.00000e+00]
 [6.59356e-01 8.48938e-01 6.57026e-01 2.15987e-01 1.00000e+00 0.00000e+00]
 [4.84302e-01 8.49623e-01 6.49632e-01 2.12752e-01 9.97812e-01 0.00000e+00]
 [7.32444e-01 8.49223e-01 6.46664e-01 2.14702e-01 9.98403e-01 0.00000e+00]
 [2.57854e-01 8.50097e-01 6.43176e-01 2.13378e-01 1.00000e+00 5.78000e-04]
 [6.59356e-01 8.51279e-01 6.44963e-01 2.16643e-01 9.98385e-01 0.00000e+00]
 [4.84302e-01 8.48283e-01 6.45067e-01 2.13123e-01 1.00000e+00 0.00000e+00]
 [3.66043e-01 8.50356e-01 6.49428e-01 2.16066e-01 9.98586e-01 0.00000e+00]
 [7.32444e-01 8.48511e-01 6.52208e-01 2.13870e-01 1.00000e+00 3.74000e-04]
 [2.57854e-01 8.52540e-01 6.58144e-01 2.17917e-01 9.96438e-01 1.36800e-03]
 [6.59356e-01 8.53160e-01

In [18]:
print("Y Train List")
for i in range(5):
    print(tModel.data_handler._y_train_list[i].shape)

print("\nY Cross Validation List")
for i in range(5):
    print(tModel.data_handler._y_crossVal_list[i].shape)
    print(tModel.data_handler._y_crossVal_list[i][0])
    print(tModel.data_handler._y_crossVal_list[i][-1])

print("\nY Test List")
for i in range(5):
    print(tModel.data_handler._y_test_list[i].shape)
    print(tModel.data_handler._y_test_list[i][0])
    print(tModel.data_handler._y_test_list[i][-1])

# self._y_train_list = list()
# self._y_crossVal_list = list()
# self._y_test_list = list()

Y Train List
(2295, 1)
(2385, 1)
(1982, 1)
(1089, 1)
(2100, 1)

Y Cross Validation List
(1, 1)
[0]
[0]
(1, 1)
[0]
[0]
(1, 1)
[0]
[0]
(1, 1)
[1]
[1]
(1, 1)
[1]
[1]

Y Test List
(1, 1)
[0]
[0]
(1, 1)
[0]
[0]
(1, 1)
[1]
[1]
(1, 1)
[0]
[0]
(1, 1)
[1]
[1]


In [19]:
tModel.data_handler.print_sequence_shapes()

Sequence length 30
Sequence stride 1
X_train len 78
X_crossVal len 26
X_test len 26
y_train len 78
y_crossVal len 26
y_test len 26
X_train[0]
(2295, 6)
[[2.57854e-01 8.50068e-01 6.52842e-01 2.12580e-01 9.99067e-01 0.00000e+00]
 [6.59356e-01 8.48786e-01 6.49076e-01 2.17237e-01 9.98857e-01 0.00000e+00]
 [4.84302e-01 8.50774e-01 6.44531e-01 2.13703e-01 9.99068e-01 9.41000e-04]
 ...
 [2.57854e-01 8.48167e-01 6.44464e-01 2.16731e-01 1.00000e+00 2.14000e-04]
 [6.59356e-01 8.48779e-01 6.44683e-01 2.12368e-01 1.00000e+00 0.00000e+00]
 [4.84302e-01 8.48439e-01 6.45801e-01 2.16505e-01 9.99250e-01 0.00000e+00]]
X_crossVal[0]
(30, 6)
[[4.84302e-01 8.49100e-01 6.50041e-01 2.15571e-01 9.99730e-01 0.00000e+00]
 [3.66043e-01 8.48995e-01 6.53369e-01 2.14565e-01 1.00000e+00 1.78400e-03]
 [7.32444e-01 8.49803e-01 6.53430e-01 2.13269e-01 1.00000e+00 0.00000e+00]
 [2.57854e-01 8.51221e-01 6.58088e-01 2.18497e-01 9.99215e-01 0.00000e+00]
 [6.59356e-01 8.48938e-01 6.57026e-01 2.15987e-01 1.00000e+00 0.00000e

In [20]:
tModel.data_handler.generate_train_data(True)

In [21]:
tModel.data_handler.generate_test_data(True)

In [ ]:
print(tModel.data_handler._y_train[0])

In [ ]:
#vHandler.load_data(cross_validation_ratio = 0.3, test_ratio = 0.2, unroll = True)
#tModel.load_data(unroll = True, verbose = 0, cross_validation_ratio = 0.2, test_ratio = 0.2)

In [22]:
vHandler.print_data(True)

#tModel = SequenceTunableModelRegression('ANN_Model', model, lib_type = 'keras', data_handler = vHandler)

Printing shapes

Training data (X, y)
(142472, 180)
(142472, 1)
Cross-Validation data (X, y)
(26, 180)
(26, 1)
Testing data (X, y)
(26, 180)
(26, 1)
Printing first 5 elements

Training data (X, y)
[[2.57854e-01 8.50068e-01 6.52842e-01 2.12580e-01 9.99067e-01 0.00000e+00
  6.59356e-01 8.48786e-01 6.49076e-01 2.17237e-01 9.98857e-01 0.00000e+00
  4.84302e-01 8.50774e-01 6.44531e-01 2.13703e-01 9.99068e-01 9.41000e-04
  3.66043e-01 8.47295e-01 6.44077e-01 2.16716e-01 1.00000e+00 0.00000e+00
  7.32444e-01 8.48956e-01 6.44328e-01 2.15185e-01 9.99924e-01 1.09300e-03
  2.57854e-01 8.48218e-01 6.46958e-01 2.13480e-01 1.00000e+00 4.05000e-04
  6.59356e-01 8.47082e-01 6.49829e-01 2.14041e-01 9.99641e-01 3.14000e-04
  4.84302e-01 8.50544e-01 6.54038e-01 2.15271e-01 9.98621e-01 2.40000e-04
  3.66043e-01 8.48957e-01 6.58088e-01 2.14787e-01 9.98100e-01 1.36000e-03
  7.32444e-01 8.49438e-01 6.55671e-01 2.15031e-01 1.00000e+00 0.00000e+00
  2.57854e-01 8.51898e-01 6.51634e-01 2.14278e-01 1.00000e+00 0

In [ ]:
broken_counter = 0
not_broken_counter = 0
lol = vHandler._y_crossVal
print(lol.shape)
#print(vHandler._X_test[1])
print(vHandler._y_test)

for i in lol:
    #print(i)
    if (i[0] == 1.0):
        not_broken_counter += 1;
    else:
        broken_counter += 1;
        
print("Broken Counter:", broken_counter)
print("Not Broken Counter:", not_broken_counter)